In [8]:
from ultralytics import YOLO
import cv2
import pytesseract
import os
import sqlite3

In [9]:
# Function to create the table if it doesn't exist
def create_table():
    # Connect to the SQLite database
    conn = sqlite3.connect("vehicle_data.db")
    # Create a cursor object to execute SQL statements
    c = conn.cursor()
    # Create the "vehicles" table with two columns: "vehicle_number" and "bike_image_path"
    # The "IF NOT EXISTS" clause ensures that the table is only created if it doesn't already exist
    c.execute("CREATE TABLE IF NOT EXISTS vehicles (vehicle_number TEXT, bike_image_path TEXT)")
    # Commit the changes to the database
    conn.commit()
    # Close the database connection
    conn.close()

In [10]:
# Function to insert a record into the "vehicles" table
def insert_record(vehicle_number, bike_image_path):
    # Connect to the SQLite database
    conn = sqlite3.connect("vehicle_data.db")
    # Create a cursor object to execute SQL statements
    c = conn.cursor()
    # Insert the record into the "vehicles" table using parameterized SQL statement
    c.execute("INSERT INTO vehicles VALUES (?, ?)", (vehicle_number, bike_image_path))
    # Commit the changes to the database
    conn.commit()
    # Close the database connection
    conn.close()

In [11]:
# Initialize YOLO models
person_bike_model = YOLO(r"Bike detectionbest.pt")
helmet_model = YOLO(r"Helmet Detection.pt")
number_plate_model = YOLO(r"Liscence Plate Detection.pt")

In [12]:
# Set up Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"E:/python/Lib/site-packages/pytesseract"  # Update with the path to your Tesseract OCR executable

In [13]:
output_dir = r"out"  # Directory to save the output images

In [14]:
# Set up video capture
video_capture = cv2.VideoCapture(0)  # Use 0 for the default camera or provide the desired camera index

In [15]:
import cv2
import os
import pytesseract

# Initialize video capture
video_capture = cv2.VideoCapture(0)  # Change to the appropriate video source if needed

# Example models (replace with your actual models)
person_bike_model = person_bike_model 
helmet_model = helmet_model 
number_plate_model = number_plate_model

# Specify the output directory
output_dir = "output_directory"

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    if not ret:
        print("Error reading frame")
        break

    # Process frame
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect person on a bike
    person_bike_results = person_bike_model.predict(img)

    # Process each detection result
    for r in person_bike_results:
        boxes = r.boxes

        # Filter detections for person on a bike
        for box in boxes:
            cls = box.cls
            print(person_bike_model.names[int(cls)], person_bike_model.names[int(cls)] == "Person_Bikes")
            if person_bike_model.names[int(cls)] == "Person_Bike":
                # Get coordinates of the bounding box
                x1, y1, x2, y2 = box.xyxy[0]

                # Draw bounding box and label on the frame
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                label = person_bike_model.names[int(cls)]
                cv2.putText(frame, label, (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Crop person on a bike image
                person_bike_image = frame[int(y1):int(y2), int(x1):int(x2)]

                # Detect helmet on the person
                helmet_results = helmet_model.predict(person_bike_image)

                # Process each helmet detection result
                for hr in helmet_results:
                    h_boxes = hr.boxes

                    # Filter detections for no helmet
                    for h_bo in h_boxes:
                        h_cls = h_bo.cls
                        if not helmet_model.names[int(h_cls)] == "With Helmet":
                            # Get coordinates of the helmet bounding box
                            h_x1, h_y1, h_x2, h_y2 = h_bo.xyxy[0]

                            # Draw bounding box and label on the frame
                            cv2.rectangle(frame, (int(x1 + h_x1), int(y1 + h_y1)),
                                          (int(x1 + h_x2), int(y1 + h_y2)), (0, 0, 255), 2)
                            h_label = helmet_model.names[int(h_cls)]
                            cv2.putText(frame, h_label, (int(x1 + h_x1), int(y1 + h_y1) - 5),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

                            # Extract number plate from the person bike image
                            number_plate_results = number_plate_model.predict(person_bike_image)

                            # Process each number plate detection result
                            for npr in number_plate_results:
                                np_boxes = npr.boxes

                                # Filter detections for number plate
                                for np_box in np_boxes:
                                    np_cls = np_box.cls
                                    print(number_plate_model.names[int(np_cls)])
                                    if number_plate_model.names[int(np_cls)] == "number_plate":
                                        # Get coordinates of the number plate bounding box
                                        np_x1, np_y1, np_x2, np_y2 = np_box.xyxy[0]

                                        # Draw bounding box and label on the frame
                                        cv2.rectangle(frame, (int(x1 + np_x1), int(y1 + np_y1)),
                                                      (int(x1 + np_x2), int(y1 + np_y2)), (255, 0, 0), 2)
                                        np_label = number_plate_model.names[int(np_cls)]
                                        cv2.putText(frame, np_label, (int(x1 + np_x1), int(y1 + np_y1) - 5),
                                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

                                        # Save the cropped number plate image
                                        output_file = f"person_violation_{some_placeholder}.png"
                                        output_path = os.path.join(output_dir, output_file)
                                        cv2.imwrite(output_path, number_plate_image)

                                        # Perform OCR on the number plate image
                                        gray = cv2.cvtColor(number_plate_image, cv2.COLOR_BGR2GRAY)
                                        text = pytesseract.image_to_string(gray)

                                        # Example usage
                                        # Create the "vehicles" table if it doesn't exist
                                        # create_table()
                                        # Insert two records into the "vehicles" table
                                        # insert_record(text, output_path)
                                        # Print the extracted text
                                        print("Number Plate Text:", text)

    # Display the frame with added information
    cv2.imshow('Person Detection', frame)

    # Wait for the user to press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
video_capture.release()
cv2.destroyAllWindows()

0: 128x224 (no detections), 209.8ms
Speed: 5.1ms preprocess, 209.8ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 224)
0: 128x224 1 Person_Bike, 99.7ms
Speed: 0.0ms preprocess, 99.7ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 224)
Person_Bike False
0: 224x192 (no detections), 114.1ms
Speed: 1.0ms preprocess, 114.1ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 192)
0: 128x224 1 Person_Bike, 114.4ms
Speed: 1.0ms preprocess, 114.4ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 224)
Person_Bike False
0: 224x224 (no detections), 103.2ms
Speed: 0.0ms preprocess, 103.2ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
0: 128x224 1 Person_Bike, 100.2ms
Speed: 0.0ms preprocess, 100.2ms inference, 0.0ms postprocess per image at shape (1, 3, 128, 224)
Person_Bike False
0: 224x224 (no detections), 95.0ms
Speed: 0.0ms preprocess, 95.0ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
0: 128x224 1 Perso